### Descargar el dataset

En nuestro caso decidimos utilizar 2 datasets adquiridos en la plataforma roboflow, siendo los siguientes:
- DETECTOR PLACAS Object Detection Model (v2, 2025-08-05 3:16pm) by CHATO. (2025, November 14). Roboflow. https://universe.roboflow.com/chato-oegqb/detector-placas-mkuhl/dataset/2

- Placas Object Detection Dataset by Xavier jimenez. (2022, April 6). Roboflow. https://universe.roboflow.com/xavier-jimenez/placas-stcpz


Estos datasets fueron descargados y combinados, guardandolos en la carpeta "dataset" que tiene sus carpetas subyacentes:

- test (dataset/test)
- train (train/test)
- valid (valid/test)


### Configuración del Entorno 
Instalación de librerías y carga de dependencias necesarias.

In [3]:
# Instalamos solo lo necesario (silencioso para no ensuciar la salida)
%pip install ultralytics easyocr opencv-python-headless matplotlib -q

import os
import cv2
import easyocr
import matplotlib.pyplot as plt
from ultralytics import YOLO

# Configuración visual para matplotlib
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)

print("✅ Entorno configurado correctamente.")


[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
✅ Entorno configurado correctamente.


### Configuración del Dataset 
Generamos el archivo de configuración data.yaml usando rutas absolutas para evitar errores de localización.

In [2]:
# Obtenemos la ruta absoluta actual para evitar errores de "path not found"
ruta_dataset = os.path.abspath("dataset")

# Contenido del archivo de configuración
contenido_yaml = f"""
path: {ruta_dataset}
train: train
val: valid
test: test

# Clases
nc: 1
names: ['placa']
"""

# Guardar archivo
with open('data.yaml', 'w') as f:
    f.write(contenido_yaml)

print(f"✅ Archivo data.yaml generado apuntando a: {ruta_dataset}")

✅ Archivo data.yaml generado apuntando a: /Users/edithriostorres/Desktop/REPOSITORISO/Topicos_IA/Unidad4/ProyectoDeteccionPlacas/ModeloDetectorPlacas/dataset


### Preprocesamiento de Datos

Convertimos las anotaciones originales del dataset al formato estándar de YOLO. Esto implica normalizar las coordenadas de los píxeles a valores entre 0 y 1 para que la red neuronal pueda procesar imágenes de cualquier tamaño.

In [3]:
import os
import cv2
import shutil

CARPETA_RAIZ = "dataset" 
CONJUNTOS = ["train", "valid", "test"]

def convertir_formato_a_yolo():
    print("Conversión de _annotations.txt a formato YOLO...")
    
    for conjunto in CONJUNTOS:
        ruta_conjunto = os.path.join(CARPETA_RAIZ, conjunto)
        ruta_txt = os.path.join(ruta_conjunto, "_annotations.txt")
        
        if not os.path.exists(ruta_txt):
            print(f" No se encontró anotaciones en {conjunto}, saltando...")
            continue
            
        print(f"📂 Procesando: {conjunto}...")
        
        with open(ruta_txt, 'r') as f:
            lineas = f.readlines()
            
        contador = 0
        for linea in lineas:
            partes = linea.strip().split()
            if not partes: continue
            
            nombre_imagen = partes[0]
            coords_raw = partes[1:]
            
            ruta_img = os.path.join(ruta_conjunto, nombre_imagen)
            

            img = cv2.imread(ruta_img)
            if img is None: continue
            h_img, w_img = img.shape[:2]
            
            contenido_yolo = []
            
            for caja in coords_raw:
                try:
                    
                    vals = list(map(float, caja.split(',')))
                    x_min, y_min, x_max, y_max, class_id = vals
                    
                    # Conversión a YOLO (x_centro, y_centro, ancho, alto) NORMALIZADO (0 a 1)
                    w_box = x_max - x_min
                    h_box = y_max - y_min
                    x_center = x_min + (w_box / 2)
                    y_center = y_min + (h_box / 2)
                    
                    x_c_norm = x_center / w_img
                    y_c_norm = y_center / h_img
                    w_norm = w_box / w_img
                    h_norm = h_box / h_img
                    
                    # Formato YOLO: clase x_c y_c w h
                    contenido_yolo.append(f"{int(class_id)} {x_c_norm:.6f} {y_c_norm:.6f} {w_norm:.6f} {h_norm:.6f}")
                except ValueError:
                    continue
            
            # Guardar archivo .txt individual
            nombre_txt = os.path.splitext(nombre_imagen)[0] + ".txt"
            ruta_destino_txt = os.path.join(ruta_conjunto, nombre_txt)
            
            with open(ruta_destino_txt, 'w') as out_f:
                out_f.write('\n'.join(contenido_yolo))
            
            contador += 1
            
        print(f"✅ {conjunto}: Se crearon {contador} archivos .txt de etiquetas.")

convertir_formato_a_yolo()

Conversión de _annotations.txt a formato YOLO...
📂 Procesando: train...
✅ train: Se crearon 507 archivos .txt de etiquetas.
📂 Procesando: valid...
✅ valid: Se crearon 105 archivos .txt de etiquetas.
📂 Procesando: test...
✅ test: Se crearon 62 archivos .txt de etiquetas.


### Entrenamiento del modelo
Utilizamos YOLOv8 Nano pre-entrenado y lo ajustamos a nuestro dataset de placas durante solo 20 épocas debido al tiempo de procesamiento.

In [6]:
# Cargar modelo base
model = YOLO('yolov8n.pt') 

# Entrenar
results = model.train(data='data.yaml', 
                      epochs=20, 
                      imgsz=640,
                      patience=20,
                      batch=16,
                      degrees=15.0,
                      perspective=0.0005,
                      )

print("✅ Entrenamiento terminado.")

Ultralytics 8.3.230 🚀 Python-3.10.10 torch-2.9.1 CPU (Apple M1)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, p

### Evaluación del modelo
Verificamos la precisión (qué tanto acierta) y el recall (cuántas placas reales encuentra) usando el set de validación.

In [4]:
# Cargar el mejor peso obtenido del entrenamiento anterior
best_model = YOLO('runs/detect/train2/weights/best.pt')


# Validar
metrics = best_model.val()

print(f"\n📊 Resultados de Validación:")
# map50 es mAP al 50%
print(f"mAP50 (Detección General): {metrics.box.map50:.2%}") 
# map es mAP50-95 (La métrica combinada)
print(f"mAP50-95 (Precisión de Caja): {metrics.box.map:.2%}") 
# mp es Mean Precision
print(f"Precision (Exactitud): {metrics.box.mp:.2%}") 
# mr es Mean Recall
print(f"Recall (Sensibilidad): {metrics.box.mr:.2%}")

Ultralytics 8.3.230 🚀 Python-3.10.10 torch-2.9.1 CPU (Apple M1)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.3±0.2 ms, read: 100.0±49.3 MB/s, size: 45.6 KB)
val: Scanning /Users/edithriostorres/Desktop/REPOSITORISO/Topicos_IA/Unidad4/ProyectoDeteccionPlacas/ModeloDetectorPlacas/dataset/valid.cache... 105 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 105/105 291.5Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 7/7 3.1s/it 21.6s3.8ss
                   all        105        108      0.991       0.97      0.991      0.824
Speed: 1.2ms preprocess, 197.1ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to /Users/edithriostorres/Desktop/REPOSITORISO/Topicos_IA/Unidad4/ProyectoDeteccionPlacas/ModeloDetectorPlacas/runs/detect/val4

📊 Resultados de Validación:
mAP50 (Detección General): 99.11%
mAP50-95 (Precisión de Caja): 82.45%
Pre

### Prueba visual final
Tomamos imágenes que la IA nunca ha visto (test) y aplicamos detección de placas + lectura de texto con EasyOCR.

In [12]:
import cv2
import easyocr
import math
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

# Cargar el modelo YOLO entrenado
ruta_modelo = 'runs/detect/train2/weights/best.pt'
model = YOLO(ruta_modelo) 

print("⏳ Cargando OCR ...")
# Inicializar OCR
reader = easyocr.Reader(['es'], gpu=False) 

cap = cv2.VideoCapture(0)

print("📷 Iniciando cámara...")

while True:
    ret, frame = cap.read()
    if not ret: break

    resultados = model(frame, stream=True, verbose=False)

    for r in resultados:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            
            confianza = math.ceil((box.conf[0]*100))/100
            
            if confianza > 0.5:
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
                
                # 2. Recortar e intentar leer
                try:
                    placa_recorte = frame[y1:y2, x1:x2]
                    texto_detectado = reader.readtext(placa_recorte)
                    
                    if len(texto_detectado) > 0:
                        texto_placa = texto_detectado[0][1]
                        
                        # Dibujar texto
                        cv2.rectangle(frame, (x1, y1-40), (x1+200, y1), (0,0,0), -1)
                        cv2.putText(frame, texto_placa.upper(), (x1, y1 - 10), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
                        
                        print(f"Placa: {texto_placa}")
                except Exception:
                    pass

    cv2.imshow('Detector Final', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Using CPU. Note: This module is much faster with a GPU.


⏳ Cargando OCR (esto puede tardar un poco la primera vez)...
Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete📷 Iniciando cámara...


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Placa: HI
Placa: HUJ 'a
Placa: IKLU-94-69
Placa: HLU-94-69
Placa: KLU-94-69
Placa: FLU-94-69
Placa: HLU-94-69
Placa: KLU-94-69
Placa: ALIJ-94-69
Placa: ALIJ-94-69
Placa: HLU-94-69
Placa: HLU-94-62
Placa: HLU-94-69
Placa: HLU-94-69
Placa: KLU-94-69
Placa: HLU-94-69
Placa: KLU-94-63
Placa: KLU-94-69
Placa: KLU-94-69
Placa: KLU-94-69
Placa: HLU-94-69
Placa: 27
Placa: co
Placa: gcogla
Placa: eom
Placa: KLU-94-69
Placa: HLU-94-69
Placa: HLU-94-69
Placa: HLU-94-69]
Placa: HLU-94-69
Placa: HLU-94-69
Placa: DVMTI
Placa: HLU-94-69
Placa: HLU-94-69
Placa: Duye
Placa: HLU-94-69
Placa: DllAyTtu
Placa: DVMIu
Placa: 8MIRA
Placa: 0
Placa: Dluyu
Placa: Dila
Placa: D
Placa: 0
Placa: CDMX
Placa: CDMX
Placa: CDMX
Placa: CDMX
Placa: CDMX
Placa: CDMX
Placa: CDMX
Placa: CDMX
Placa: CDMX
Placa: CDMX
Placa: CDMX
Placa: CDMX
Placa: CDMX
Placa: CDMX
Placa: CDMX
Placa: CDM 
Placa: KLU-94-69
Placa: 94-69
Placa: CDMX
Placa: CDMX
Placa: CDMX
Placa: PLB
Placa: [GR8 PL8
Placa: GR8 PL8
Placa: GR8 PL8
Placa: GR8 PL8
Pl

: 